### Librerias a usar

In [3]:
import pandas as pd

import math
import re, os, csv, sys, time, fnmatch, shutil
from os import chdir, getcwd, path
import json, random, argparse
import io, pickle, codecs
import networkx as nx, builtins
from shutil import copyfile
import time

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.options import Options
from selenium.common.exceptions import TimeoutException, NoSuchElementException

In [2]:
amigos_iteracion = 80 # Cuántos "amigos" por cada iteración
pausa_cambios = 55 # Minutos pausados para cambiar de usuario y no se bloqueen los perfiles de facebook

#### Funciones para scrapear en facebook

In [3]:
# pausa por cada amigo
def pause(min=2,max=5):
    return round(random.uniform(min,max),1)

In [4]:
# login
def do_login(driver,usr,pwd):
    driver.get('https://www.facebook.com')
    try:
        elem = driver.find_element_by_id('email')
        elem.send_keys(usr)
        elem = driver.find_element_by_id('pass')
        elem.send_keys(pwd)
    except NoSuchElementException:
        elem = driver.find_element_by_name('email')
        elem.send_keys(usr)
        elem = driver.find_element_by_name('pass')
        elem.send_keys(pwd)
    except Exception as ex:
        print('[!] Error:')
        print(ex)
        sys.exit(0)
    elem.send_keys(Keys.RETURN)
    time.sleep(pause(5,8))

In [5]:
# obtiene información del target
def get_target_info_new(target,driver):
    driver.get('https://www.facebook.com/%s' % target)
    time.sleep(pause())
    try:
        name = driver.find_element_by_xpath("//h1[@class='gmql0nx0 l94mrbxd p1ri9a11 lzcic4wl bp9cbjyn j83agx80']").text
    except:
        try:
            name = re.findall('<title>([^"]+)</title>',driver.page_source)[0]
        except:
            name = 'TARGET'
    return [name,driver.current_url]

In [6]:
# busca amigos en común y devuelve lista friends con id, nombre y url.
def get_friends(driver,usr1,usr2,url='https://www.facebook.com/browse/mutual_friends/?uid=%s&node=%s'):
    friends = []
    final_url = url % (usr1,usr2)
    # intentos
    attempts = 0
    while attempts < 5:
        try:
            driver.get(final_url)
            break
        except TimeoutException as ex:
            print('[!] TimeoutException. Attempt #%d' % attempts)
            attempts +=1
            pause(attempts*10,attempts*10)

    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
        time.sleep(pause(1,2))
    
    try:
        users = driver.find_elements_by_css_selector("div[class='fsl fwb fcb']")
        if len(users) == 0:
            friend = {}
            friend['name'] = []
            friend['id'] = []
            friend['url'] = []
            friends.append(friend)
        else:
            for user in users:
                s = user.get_attribute('innerHTML')
                try:
                    uid = re.findall('/ajax/hovercard/user.php\?id=([0-9]*)(?:"|&amp;)',s)
                    url = re.findall('href=[\'"]?([^\'" >]+)',s)
                    name = re.findall('>[.*]?([^<]+)',s)
                except:
                    print('[!] Error! Cannot read data from HTML page')
                    continue
                friend = {}
                try:
                    friend['name'] = name[0].encode("utf-8", errors="replace").decode("utf-8", errors="replace")
                except:
                    friend['name'] = 'decoding error'
                friend['id'] = int(uid[0])
                friend['url'] = re.sub('(\?|\&amp.+)fref.*','',url[0])
                friends.append(friend)

    except Exception as ex:
        print('[!] Error! %s' % ex)
        raise
    
    return friends

In [7]:
# Obteniendo amigos de calificado o clientes desde los apóstoles scrapeados
def get_friends_calf_apost(id_target,apostoles_list,driver):
    friends_with_calf_apost = []
    counter = 0
    # Inicia contador interno de iteración
    for idP in apostoles_list:
        time.sleep(pause())
        counter+=1
        if counter == amigos_iteracion:
            print('Pausa por %d minutos' % (pausa_cambios))
            print(time.strftime("%H:%M:%S"))
            time.sleep(pausa_cambios * 60)
            counter = 0
#       Esta segunda opción de pausa se da si la lista de apóstoles no es igual a la de amigos_iteracion
        elif counter == len(apostoles_list):
            print('Pausa por %d minutos segunda opción de pausa' % (pausa_cambios))
            print(time.strftime("%H:%M:%S"))
            time.sleep(pausa_cambios * 60)
            counter = 0
#         count_new = count_old = 0
        friends_in_comun = get_friends(driver, id_target,idP)
        friends_in_comun_df = pd.DataFrame(friends_in_comun)
        friend= {}
        friend['id_Calf'] = id_target
        friend['name_Calf'] = name_target
        friend['type_Calf'] = type_target
        friend['id_Apost'] = idP
        friend['ids_friends'] = friends_in_comun_df['id'].tolist()
        friend['names_friends'] = friends_in_comun_df['name'].tolist()
        friend['urls_friends'] = friends_in_comun_df['url'].tolist()
        friends_with_calf_apost.append(friend)
    driver.close()
    return friends_with_calf_apost

#### obteniendo la lista de apostoles

In [8]:
file_name =  'Apostoles_LCM.xlsx'
df_apostoles = pd.read_excel(io=file_name)

In [9]:
df_apostoles = df_apostoles[df_apostoles['amigos grafo']> 30]
df_apostoles = df_apostoles
df_apostoles['facebook id'] = df_apostoles['facebook id'].astype(str)

In [10]:
apostoles_list = df_apostoles['facebook id'].tolist()

In [11]:
len(apostoles_list)

364

#### Lista de claves

In [12]:
df_claves = pd.read_excel(io='claves.xlsx')

#### Obteniendo personas a scrapear

In [4]:
file_name_to_scrapping =  'calificados_clientes.xlsx'
df_lista_a_scrapear = pd.read_excel(io=file_name_to_scrapping)

In [5]:
# función que establece el tipo de contacto
def set_type(ESTADO,potencial):
    type_potencial = ''
    if ESTADO == 'CLB':
        return 'Cliente'
    else:
        if 'Medio' in potencial:
            type_potencial = 'Medio'
        elif 'Muy Alto' in potencial:
            type_potencial = 'Muy Alto'
        elif 'Alto' in potencial:
            type_potencial = 'Alto'
        return type_potencial

In [6]:
df_lista_a_scrapear['TIPO'] = df_lista_a_scrapear.apply(lambda x: set_type(x['ESTADO'],x['POTENCIAL']), axis=1)

### Número de iteraciones

In [16]:
# obteniendo número de iteraciones
num_ite = int(math.ceil(len(apostoles_list)/amigos_iteracion))

# función que divide la lista de apostoles según el número de iteraciones
def split_apost(my_list,num_ite):
    avg = len(my_list) / float(num_ite)
    out = []
    last = 0.0

    while last < len(my_list):
        out.append(my_list[int(last):int(last + avg)])
        last += avg
    return out

In [17]:
# dividiendo la lista de apóstoles en el número de listas según el número de iteración
apostoles_split = split_apost(apostoles_list,num_ite)

In [18]:
len(apostoles_split)

5

In [19]:
# apostoles_split

### Iteraciones

In [20]:
# función que itera según el número de listas divididas y cambia de usuario de facebook
def ite_num(apostoles_split, df_claves,id_target, name_target, type_target):
    appended_data = []
    j=0
    
    # Archivo csv de amigos
    csv_file_path = '%d-amigos_con_apostoles.csv' % id_target
    csv_file_path_grouped = '%d-amigos_agrupados.csv' % id_target
    
    for apostoles_sub_list in apostoles_split:
        usr = df_claves.loc[j,'email']
        pwd = df_claves.loc[j,'password']
#         print(apostoles_sub_list)
        print(usr)

        # Seteo inicial
        options = Options()
        driver = webdriver.Firefox(options=options)
        start = time.time()

        # Login
        do_login(driver,usr,pwd)

        # Ejecución de funciones
        target_name,target_url = get_target_info_new(id_target,driver)

        nombre_fb = target_name
        print('\nFetching friends of target user "%s" (%d)...... correo: "%s"...\n' % (nombre_fb, id_target,usr))
        
        # buscando amigos en común solo de la sublista de apóstoles
        friends = get_friends_calf_apost(id_target,apostoles_sub_list,driver)
        df_friends_in_comun_cal_apost = pd.DataFrame(friends)
        

        print('\n')
        print(time.strftime("%H:%M:%S"))
        print('\nFin!!! Se agregaron los amigos en común a cada target')
        if j == len(df_claves) - 1:
            j = 0
        else:
            j = j+1
        appended_data.append(df_friends_in_comun_cal_apost)
        
    appended_data = pd.concat(appended_data)
    appended_data.to_csv(csv_file_path,  index=False)
    
    # preparación de dataframe para agrupar en una sola fila
    aggregation_functions = {'id_Calf': 'first', 'name_Calf': 'first', 'type_Calf': 'first','id_Apost': 'sum', 'ids_friends': 'sum', 
                         'names_friends': 'sum', 'urls_friends': 'sum'}
    # uniendo df a una sola fila
    new_df_friends = df_friends_in_comun_cal_apost.groupby(df_friends_in_comun_cal_apost['id_Calf']).aggregate(aggregation_functions)
    new_df_friends = new_df_friends.reset_index(drop=True)

    #rename 
    new_df_friends.rename(columns={'id_Calf':'id_Cal'}, inplace = True)

    # eliminando columna que contiene ids de los apóstoles
    new_df_friends.drop(['id_Apost'], axis='columns', inplace=True)

    new_df_friends.to_csv(csv_file_path_grouped, index=False)

    # crear carpeta del target
    def create_folder(folder):
        if not os.path.exists(folder):
            os.mkdir(folder)

    nombre_carpeta = '%s (id %d)' % (name_target, id_target)
    create_folder(nombre_carpeta)

    # listado de archivos a mover
    lista_total = list(os.listdir('.'))
    extension1 = str(id_target) + '-*'
    lista_final = pd.DataFrame(data=None,columns=['nombre'])
    for archivo in lista_total:
        if fnmatch.fnmatch(archivo, extension1):
            new_entry = {'nombre': archivo}
            lista_final.loc[len(lista_final)] = new_entry

    # mover los files a su carpeta respectiva y crear una copia del archivo que contienen los ids en 
    # una sola fila y mover la copia a carpeta que contiene el file de todos los calificados ya scrapeados 
    # llamada amigos_calificados
    for i in range(len(lista_final)):
        if 'agrupados' in lista_final.loc[i,'nombre']:
            copyfile(lista_final.loc[i,'nombre'], 'amigos_calificados/'+'todos-'+nombre_carpeta+'.csv')
        origen = lista_final.loc[i,'nombre']
        destino = nombre_carpeta + "/" + origen
        shutil.move(origen, destino)

    # mover carpeta del target a carpeta personas_scrapeadas
    original = nombre_carpeta
    target = 'personas_scrapeadas'
    shutil.move(original,target)
    return(appended_data, new_df_friends)

In [7]:
# Recomendación: avanzar por partes 

# LC  me quedé en SANCHEZ AGUILAR DENISE OFELIA, estos serían los 5 siguientes a scrapear
targets_5 = df_lista_a_scrapear.iloc[24:28]

In [9]:
# iteración entre los targets
finish_result = []
count = 0
for index, row in targets_5.iterrows():
    id_target = row['PROSPECTO_ID_FB']
    name_target = row['NOMBRE1']
    type_target = row['TIPO']
    print('\nSon %d personas a scrapear, vas scrapeando %d' % ((len(targets_5)),count+1 ))
    count = count+1
    result_target = ite_num(apostoles_split, df_claves,id_target, name_target, type_target)
    finish_result.append(result_target)
    time.sleep(300)
    print('Pausa de 5 minutos antes de iniciar con otra cuenta de facebook')

In [ ]:
finish_result